# Laboratorio 1

In [3]:
!pip install pendulum==2.0.3
!pip install prefect

Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
  Created wheel for pendulum: filename=pendulum-2.0.3-cp310-cp310-win_amd64.whl size=103757 sha256=e26e8da7abc1fb8fed6dbaf83ae1c487e2ac36e5df34045abf97005a826cdf10
  Stored in directory: C:\Users\LUIGGI\AppData\Local\Temp\pip-ephem-wheel-cache-xllkp_mr\wheels\df\20\07\f2f2eb2da24589a5964e7191a567ebefa01659e2f4ff3d176c
Successfully built pendulum


You should consider upgrading via the 'D:\BCP\ALGORTIMO\ENTORNO_ETL_CERTUS\Scripts\python.exe -m pip install --upgrade pip' command.


Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
Failed to build pendulum


  ERROR: Command errored out with exit status 1:
   command: 'D:\BCP\ALGORTIMO\ENTORNO_ETL_CERTUS\Scripts\python.exe' 'D:\BCP\ALGORTIMO\ENTORNO_ETL_CERTUS\lib\site-packages\pip\_vendor\pep517\in_process\_in_process.py' build_wheel 'C:\Users\LUIGGI\AppData\Local\Temp\tmpmdc1d387'
       cwd: C:\Users\LUIGGI\AppData\Local\Temp\pip-install-ohwbir4f\pendulum_8c4a7690663f4256bbc3523456703796
  Complete output (28 lines):
  Traceback (most recent call last):
    File "C:\Users\LUIGGI\AppData\Local\Temp\pip-install-ohwbir4f\pendulum_8c4a7690663f4256bbc3523456703796\build.py", line 5, in <module>
      from distutils.command.build_ext import build_ext
    File "D:\BCP\ALGORTIMO\ENTORNO_ETL_CERTUS\lib\site-packages\_distutils_hack\__init__.py", line 92, in create_module
      return importlib.import_module('setuptools._distutils')
    File "C:\Python310\lib\importlib\__init__.py", line 126, in import_module
      return _bootstrap._gcd_import(name[level:], package, level)
  ModuleNotFoundError:

In [ ]:
#En caso exista un error https://www.sqlite.org/download.html les enseñare este tutorial

# Introduccion del flujo de prefect

Comencemos con lo básico, definiendo los componentes centrales de los flujos de trabajo de Prefect.

Un flujo es la base de todos los flujos de trabajo de Prefect. Un flujo es una función de Python decorada con un decorador. @flow

Una tarea es una función de Python decorada con un decorador. Las tareas representan distintas piezas de trabajo ejecutadas dentro de un flujo. @task

Todos los flujos de trabajo de Prefect se definen en el contexto de un flujo. Cada flujo de trabajo de Prefect debe contener al menos una función de flujo que sirva como punto de entrada para la ejecución del flujo.

Los flujos pueden incluir llamadas a tareas, así como flujos secundarios, que llamamos "subflujos" en este contexto. En un nivel alto, esto es como escribir cualquier otra aplicación de Python: organiza el trabajo específico y repetitivo en tareas y llama a esas tareas desde flujos.

In [21]:
#FLujo basico
from prefect import flow
import sqlite3
@flow
def my_favorite_function():
    print("What is your favorite number?")
    return(42)

my_favorite_function()

20:06:44.589 | INFO    | prefect.engine - Created flow run 'messy-fossa' for flow 'my-favorite-function'

What is your favorite number?


20:06:44.876 | INFO    | Flow run 'messy-fossa' - Finished in state Completed()

42

### Ejecutar flujos con parámetros
Al igual que con cualquier función de Python, puede pasar argumentos. Los argumentos posicionales y de palabras clave definidos en la función de flujo se denominan parámetros. Para demostrarlo, ejecute este código:

In [22]:
import requests
from prefect import flow

@flow
def call_api(url):
    return requests.get(url).json()

api_result = call_api("http://time.jsontest.com/")
print(api_result)

20:34:41.880 | INFO    | prefect.engine - Created flow run 'esoteric-raven' for flow 'call-api'

20:34:43.021 | INFO    | Flow run 'esoteric-raven' - Finished in state Completed()

{'date': '01-13-2023', 'milliseconds_since_epoch': 1673573680416, 'time': '01:34:40 AM'}


### Ejecutar un flujo básico con tareas
Agreguemos ahora algunas tareas a un flujo para que podamos orquestar y monitorear a un nivel más granular.

Una tarea es una función que representa una pieza distinta de trabajo ejecutada dentro de un flujo. No tiene que usar tareas: puede incluir toda la lógica de su flujo de trabajo dentro del propio flujo. Sin embargo, encapsular la lógica empresarial en unidades de tareas más pequeñas le proporciona una observabilidad más granular, control sobre cómo se ejecutan tareas específicas (aprovechando potencialmente la ejecución paralela) y la capacidad de reutilizar tareas en flujos y subflujos.

La creación y adición de tareas sigue exactamente el mismo patrón que para los flujos. Importe y utilice el decorador @task para anotar funciones como tareas.task

Tomemos el ejemplo anterior y movamos la solicitud HTTP real a su propia tarea.call_api()

In [23]:
import requests
from prefect import flow, task

@task
def call_api(url):
    response = requests.get(url)
    print(response.status_code)
    return response.json()

@flow
def api_flow(url):
    fact_json = call_api(url)
    return fact_json

print(api_flow("https://catfact.ninja/fact"))

20:36:56.181 | INFO    | prefect.engine - Created flow run 'qualified-ostrich' for flow 'api-flow'

20:36:56.375 | INFO    | Flow run 'qualified-ostrich' - Created task run 'call_api-ded10bed-0' for task 'call_api'

20:36:56.377 | INFO    | Flow run 'qualified-ostrich' - Executing 'call_api-ded10bed-0' immediately...

200


20:36:56.950 | INFO    | Task run 'call_api-ded10bed-0' - Finished in state Completed()

20:36:56.991 | INFO    | Flow run 'qualified-ostrich' - Finished in state Completed()

{'fact': 'A cat will tremble or shiver when it is in extreme pain.', 'length': 56}


Y, por supuesto, podemos crear tareas que tomen entradas y pasen entradas a otras tareas.

In [24]:
import requests
from prefect import flow, task

@task
def call_api2(url):
    response = requests.get(url)
    print(response.status_code)
    return response.json()

@task
def parse_fact2(response):
    fact = response["fact"]
    print(fact)
    return fact

@flow
def api_flow2(url):
    fact_json = call_api2(url)
    fact_text = parse_fact2(fact_json)
    return fact_text

api_flow2("https://catfact.ninja/fact")

D:\BCP\ALGORTIMO\Curso_ETL_Python\lib\site-packages\prefect\tasks.py:256: UserWarning: A task named 'call_api2' and defined at 'C:\Users\LUIGGI\AppData\Local\Temp\ipykernel_17924\407617822.py:4' conflicts with another task. Consider specifying a unique `name` parameter in the task definition:

 `@task(name='my_unique_name', ...)`
  warnings.warn(
D:\BCP\ALGORTIMO\Curso_ETL_Python\lib\site-packages\prefect\tasks.py:256: UserWarning: A task named 'parse_fact2' and defined at 'C:\Users\LUIGGI\AppData\Local\Temp\ipykernel_17924\407617822.py:10' conflicts with another task. Consider specifying a unique `name` parameter in the task definition:

 `@task(name='my_unique_name', ...)`
  warnings.warn(
D:\BCP\ALGORTIMO\Curso_ETL_Python\lib\site-packages\prefect\flows.py:206: UserWarning: A flow named 'api-flow2' and defined at 'C:\Users\LUIGGI\AppData\Local\Temp\ipykernel_17924\407617822.py:16' conflicts with another flow. Consider specifying a unique `name` parameter in the flow definition:

 `@

20:39:05.466 | INFO    | prefect.engine - Created flow run 'fanatic-echidna' for flow 'api-flow2'

20:39:05.733 | INFO    | Flow run 'fanatic-echidna' - Created task run 'call_api2-26498c32-0' for task 'call_api2'

20:39:05.735 | INFO    | Flow run 'fanatic-echidna' - Executing 'call_api2-26498c32-0' immediately...

200


...

20:39:06.287 | INFO    | Flow run 'fanatic-echidna' - Executing 'parse_fact2-3fba20db-0' immediately...

The leopard is the most widespread of all big cats.


20:39:06.413 | INFO    | Task run 'parse_fact2-3fba20db-0' - Finished in state Completed()

20:39:06.475 | INFO    | Flow run 'fanatic-echidna' - Finished in state Completed()

'The leopard is the most widespread of all big cats.'

### Ejecutar un flujo dentro de un flujo
¡No solo puede llamar a funciones de tareas dentro de un flujo, sino que también puede llamar a otras funciones de flujo! Los flujos secundarios se denominan subflujos y permiten administrar, realizar un seguimiento y crear versiones de la lógica multitarea común de manera eficaz. Consulte la sección Redacción de flujos de la documentación de flujos para obtener más información.

Considere el siguiente ejemplo simple:

In [25]:
from prefect import flow

@flow
def common_flow(config: dict):
    print("I am a subgraph that shows up in lots of places!")
    intermediate_result = 42
    return intermediate_result

@flow
def main_flow():
    # do some things
    # then call another flow function
    data = common_flow(config={})
    # do more things

main_flow()

20:42:25.779 | INFO    | prefect.engine - Created flow run 'eggplant-newt' for flow 'main-flow'

20:42:26.049 | INFO    | Flow run 'eggplant-newt' - Created subflow run 'careful-termite' for flow 'common-flow'

I am a subgraph that shows up in lots of places!


20:42:26.159 | INFO    | Flow run 'careful-termite' - Finished in state Completed()

20:42:26.195 | INFO    | Flow run 'eggplant-newt' - Finished in state Completed('All states completed.')

[Completed(message=None, type=COMPLETED, result=42)]

### Configuración básica de flujo
La configuración básica del flujo incluye la capacidad de proporcionar un nombre, una descripción y una versión y otras opciones para el flujo mediante argumentos de flujo.

La configuración de flujo se especifica como argumentos en el decorador.@flow

Un flujo es una pieza distinguida de metadatos dentro de Prefect. El nombre que se asigna a un flujo se convierte en el identificador unificador de todas las ejecuciones futuras de ese flujo, independientemente de la versión o la estructura de tareas.name

In [31]:
from prefect import flow, task

@task(name="My Example Task", 
      description="An example task for a tutorial.",
      tags=["tutorial","tag-test"])
def my_task():
    print("hello world")

@flow
def my_flow():
    my_task()
    
my_flow()

D:\BCP\ALGORTIMO\Curso_ETL_Python\lib\site-packages\prefect\tasks.py:256: UserWarning: A task named 'My Example Task' and defined at 'C:\Users\LUIGGI\AppData\Local\Temp\ipykernel_17924\298959815.py:3' conflicts with another task. Consider specifying a unique `name` parameter in the task definition:

 `@task(name='my_unique_name', ...)`
  warnings.warn(
D:\BCP\ALGORTIMO\Curso_ETL_Python\lib\site-packages\prefect\flows.py:206: UserWarning: A flow named 'my-flow' and defined at 'C:\Users\LUIGGI\AppData\Local\Temp\ipykernel_17924\298959815.py:9' conflicts with another flow. Consider specifying a unique `name` parameter in the flow definition:

 `@flow(name='my_unique_name', ...)`
  warnings.warn(


20:46:55.471 | INFO    | prefect.engine - Created flow run 'pistachio-lionfish' for flow 'my-flow'

20:46:55.764 | INFO    | Flow run 'pistachio-lionfish' - Created task run 'My Example Task-20c6ece6-0' for task 'My Example Task'

20:46:55.770 | INFO    | Flow run 'pistachio-lionfish' - Executing 'My Example Task-20c6ece6-0' immediately...

hello world


20:46:55.902 | INFO    | Task run 'My Example Task-20c6ece6-0' - Finished in state Completed()

20:46:55.945 | INFO    | Flow run 'pistachio-lionfish' - Finished in state Completed('All states completed.')

[Completed(message=None, type=COMPLETED, result=LiteralResult(type='literal', value=None))]

### Reintentos de flujo y tareas
Prefect incluye compatibilidad integrada para reintentos de flujo y tareas, que se configuran en el flujo o la tarea. Esto permite que los flujos y las tareas se reintenten automáticamente en caso de error. Puede especificar cuántos reintentos desea intentar y, opcionalmente, un retraso entre los intentos de reintento:

In [1]:
from prefect import flow, task

# this tasks runs 3 times before the flow fails
@task(retries=2, retry_delay_seconds=60)
def failure():
    print('running')
    raise ValueError("bad code")

@flow
def test_retries():
    return failure()

In [19]:
state = test_retries()


18:08:35.661 | INFO    | prefect.engine - Created flow run 'mustard-chimpanzee' for flow 'test-retries'

18:08:35.833 | INFO    | Flow run 'mustard-chimpanzee' - Created task run 'failure-f8b6b21b-0' for task 'failure'

18:08:35.834 | INFO    | Flow run 'mustard-chimpanzee' - Executing 'failure-f8b6b21b-0' immediately...

running


18:08:35.895 | ERROR   | Task run 'failure-f8b6b21b-0' - Encountered exception during execution:
Traceback (most recent call last):
  File "D:\BCP\ALGORTIMO\Curso_ETL_Python\lib\site-packages\prefect\engine.py", line 1445, in orchestrate_task_run
    result = await run_sync(task.fn, *args, **kwargs)
  File "D:\BCP\ALGORTIMO\Curso_ETL_Python\lib\site-packages\prefect\utilities\asyncutils.py", line 156, in run_sync_in_interruptible_worker_thread
    tg.start_soon(
  File "D:\BCP\ALGORTIMO\Curso_ETL_Python\lib\site-packages\anyio\_backends\_asyncio.py", line 662, in __aexit__
    raise exceptions[0]
  File "D:\BCP\ALGORTIMO\Curso_ETL_Python\lib\site-packages\anyio\to_thread.py", line 31, in run_sync
    return await get_asynclib().run_sync_in_worker_thread(
  File "D:\BCP\ALGORTIMO\Curso_ETL_Python\lib\site-packages\anyio\_backends\_asyncio.py", line 937, in run_sync_in_worker_thread
    return await future
  File "D:\BCP\ALGORTIMO\Curso_ETL_Python\lib\site-packages\anyio\_backends\_asyncio.py", line 867, in run
    result = context.run(func, *args)
  File "D:\BCP\ALGORTIMO\Curso_ETL_Python\lib\site-packages\prefect\utilities\asyncutils.py", line 135, in capture_worker_thread_and_result
    result = __fn(*args, **kwargs)
  File "C:\Users\LUIGGI\AppData\Local\Temp\ipykernel_17924\2081695652.py", line 7, in failure
    raise ValueError("bad code")
ValueError: bad code

...

18:10:36.154 | ERROR   | Task run 'failure-f8b6b21b-0' - Encountered exception during execution:
Traceback (most recent call last):
  File "D:\BCP\ALGORTIMO\Curso_ETL_Python\lib\site-packages\prefect\engine.py", line 1445, in orchestrate_task_run
    result = await run_sync(task.fn, *args, **kwargs)
  File "D:\BCP\ALGORTIMO\Curso_ETL_Python\lib\site-packages\prefect\utilities\asyncutils.py", line 156, in run_sync_in_interruptible_worker_thread
    tg.start_soon(
  File "D:\BCP\ALGORTIMO\Curso_ETL_Python\lib\site-packages\anyio\_backends\_asyncio.py", line 662, in __aexit__
    raise exceptions[0]
  File "D:\BCP\ALGORTIMO\Curso_ETL_Python\lib\site-packages\anyio\to_thread.py", line 31, in run_sync
    return await get_asynclib().run_sync_in_worker_thread(
  File "D:\BCP\ALGORTIMO\Curso_ETL_Python\lib\site-packages\anyio\_backends\_asyncio.py", line 937, in run_sync_in_worker_thread
    return await future
  File "D:\BCP\ALGORTIMO\Curso_ETL_Python\lib\site-packages\anyio\_backends\_asyncio.py", line 867, in run
    result = context.run(func, *args)
  File "D:\BCP\ALGORTIMO\Curso_ETL_Python\lib\site-packages\prefect\utilities\asyncutils.py", line 135, in capture_worker_thread_and_result
    result = __fn(*args, **kwargs)
  File "C:\Users\LUIGGI\AppData\Local\Temp\ipykernel_17924\2081695652.py", line 7, in failure
    raise ValueError("bad code")
ValueError: bad code

18:10:36.184 | ERROR   | Task run 'failure-f8b6b21b-0' - Finished in state Failed('Task run encountered an exception: ValueError: bad code\n')

18:10:36.186 | ERROR   | Flow run 'mustard-chimpanzee' - Encountered exception during execution:
Traceback (most recent call last):
  File "D:\BCP\ALGORTIMO\Curso_ETL_Python\lib\site-packages\prefect\engine.py", line 636, in orchestrate_flow_run
    result = await run_sync(flow_call)
  File "D:\BCP\ALGORTIMO\Curso_ETL_Python\lib\site-packages\prefect\utilities\asyncutils.py", line 156, in run_sync_in_interruptible_worker_thread
    tg.start_soon(
  File "D:\BCP\ALGORTIMO\Curso_ETL_Python\lib\site-packages\anyio\_backends\_asyncio.py", line 662, in __aexit__
    raise exceptions[0]
  File "D:\BCP\ALGORTIMO\Curso_ETL_Python\lib\site-packages\anyio\to_thread.py", line 31, in run_sync
    return await get_asynclib().run_sync_in_worker_thread(
  File "D:\BCP\ALGORTIMO\Curso_ETL_Python\lib\site-packages\anyio\_backends\_asyncio.py", line 937, in run_sync_in_worker_thread
    return await future
  File "D:\BCP\ALGORTIMO\Curso_ETL_Python\lib\site-packages\anyio\_backends\_asyncio.py", line 867, in run
    result = context.run(func, *args)
  File "D:\BCP\ALGORTIMO\Curso_ETL_Python\lib\site-packages\prefect\utilities\asyncutils.py", line 135, in capture_worker_thread_and_result
    result = __fn(*args, **kwargs)
  File "C:\Users\LUIGGI\AppData\Local\Temp\ipykernel_17924\2081695652.py", line 11, in test_retries
    return failure()
  File "D:\BCP\ALGORTIMO\Curso_ETL_Python\lib\site-packages\prefect\tasks.py", line 436, in __call__
    return enter_task_run_engine(
  File "D:\BCP\ALGORTIMO\Curso_ETL_Python\lib\site-packages\prefect\engine.py", line 926, in enter_task_run_engine
    return run_async_from_worker_thread(begin_run)
  File "D:\BCP\ALGORTIMO\Curso_ETL_Python\lib\site-packages\prefect\utilities\asyncutils.py", line 177, in run_async_from_worker_thread
    return anyio.from_thread.run(call)
  File "D:\BCP\ALGORTIMO\Curso_ETL_Python\lib\site-packages\anyio\from_thread.py", line 49, in run
    return asynclib.run_async_from_thread(func, *args)
  File "D:\BCP\ALGORTIMO\Curso_ETL_Python\lib\site-packages\anyio\_backends\_asyncio.py", line 970, in run_async_from_thread
    return f.result()
  File "C:\Users\LUIGGI\AppData\Local\Programs\Python\Python38\lib\concurrent\futures\_base.py", line 444, in result
    return self.__get_result()
  File "C:\Users\LUIGGI\AppData\Local\Programs\Python\Python38\lib\concurrent\futures\_base.py", line 389, in __get_result
    raise self._exception
  File "D:\BCP\ALGORTIMO\Curso_ETL_Python\lib\site-packages\prefect\engine.py", line 1064, in get_task_call_return_value
    return await future._result()
  File "D:\BCP\ALGORTIMO\Curso_ETL_Python\lib\site-packages\prefect\futures.py", line 237, in _result
    return await final_state.result(raise_on_failure=raise_on_failure, fetch=True)
  File "D:\BCP\ALGORTIMO\Curso_ETL_Python\lib\site-packages\prefect\states.py", line 89, in _get_state_result
    raise await get_state_exception(state)
  File "D:\BCP\ALGORTIMO\Curso_ETL_Python\lib\site-packages\prefect\engine.py", line 1445, in orchestrate_task_run
    result = await run_sync(task.fn, *args, **kwargs)
  File "D:\BCP\ALGORTIMO\Curso_ETL_Python\lib\site-packages\prefect\utilities\asyncutils.py", line 156, in run_sync_in_interruptible_worker_thread
    tg.start_soon(
  File "D:\BCP\ALGORTIMO\Curso_ETL_Python\lib\site-packages\anyio\_backends\_asyncio.py", line 662, in __aexit__
    raise exceptions[0]
  File "D:\BCP\ALGORTIMO\Curso_ETL_Python\lib\site-packages\anyio\to_thread.py", line 31, in run_sync
    return await get_asynclib().run_sync_in_worker_thread(
  File "D:\BCP\ALGORTIMO\Curso_ETL_Python\lib\site-packages\anyio\_backends\_asyncio.py", line 937, in run_sync_in_worker_thread
    return await future
  File "D:\BCP\ALGORTIMO\Curso_ETL_Python\lib\site-packages\anyio\_backends\_asyncio.py", line 867, in run
    result = context.run(func, *args)
  File "D:\BCP\ALGORTIMO\Curso_ETL_Python\lib\site-packages\prefect\utilities\asyncutils.py", line 135, in capture_worker_thread_and_result
    result = __fn(*

18:10:36.221 | ERROR   | Flow run 'mustard-chimpanzee' - Finished in state Failed('Flow run encountered an exception. ValueError: bad code\n')

ValueError: bad code

In [ ]:
#Revisemos el servidor desde el pycharm
prefect orion start

In [ ]:
#pruebas